In [1]:
import numpy as np
import pandas as pd
import joblib
from itertools import combinations
from tqdm import tqdm

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score

import warnings
warnings.filterwarnings('ignore')


In [2]:
# Load feature importance data
feature_importance_df = pd.read_csv('feature_importance_v2_baseline.csv')
print(f"Loaded feature importance data: {len(feature_importance_df)} rows")

# Create feature name to index mapping
keypoint_names = [
    'nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner', 'right_eye', 'right_eye_outer',
    'left_ear', 'right_ear', 'mouth_left', 'mouth_right', 'left_shoulder', 'right_shoulder',
    'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky',
    'left_index', 'right_index', 'left_thumb', 'right_thumb', 'left_hip', 'right_hip',
    'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel', 'right_heel',
    'left_foot_index', 'right_foot_index'
]

feature_to_index = {}
for i, name in enumerate(keypoint_names):
    feature_to_index[f'{name}_mean_x'] = (i, 'mean_x')
    feature_to_index[f'{name}_mean_y'] = (i, 'mean_y')
    feature_to_index[f'{name}_var_x'] = (i, 'var_x')
    feature_to_index[f'{name}_var_y'] = (i, 'var_y')

# Dynamic feature names and their indices (they come after the 132 static features)
dynamic_feature_names = [
    'left_side_mean_velocity', 'left_side_std_velocity', 'left_side_max_velocity',
    'right_side_mean_velocity', 'right_side_std_velocity', 'right_side_max_velocity',
    'left_side_mean_acceleration', 'left_side_std_acceleration', 'left_side_max_acceleration',
    'right_side_mean_acceleration', 'right_side_std_acceleration', 'right_side_max_acceleration',
    'velocity_mean_asymmetry_ratio', 'velocity_std_asymmetry_ratio', 'velocity_max_asymmetry_ratio',
    'acceleration_mean_asymmetry_ratio', 'acceleration_std_asymmetry_ratio', 'acceleration_max_asymmetry_ratio'
]

# Add dynamic features to the mapping (they start at index 132)
for i, name in enumerate(dynamic_feature_names):
    feature_to_index[name] = (132 + i, 'dynamic')

Loaded feature importance data: 775 rows


In [3]:
# Define left and right side keypoints
LEFT_SIDE_KEYPOINTS = [11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]  # Left body parts
RIGHT_SIDE_KEYPOINTS = [12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32]  # Right body parts

def calculate_velocity(sequence):
    """Calculate velocity between consecutive frames"""
    # sequence shape: (seq_length, 33, 2)
    velocity = np.diff(sequence, axis=0)  # (seq_length-1, 33, 2)
    return velocity

def calculate_side_velocity_stats(velocity_sequence, side_keypoints):
    """Calculate velocity statistics for a body side"""
    # velocity_sequence shape: (seq_length-1, 33, 2)
    side_velocity = velocity_sequence[:, side_keypoints, :]  # (seq_length-1, n_keypoints, 2)
    
    # Calculate magnitude of velocity vectors
    velocity_magnitudes = np.sqrt(np.sum(side_velocity**2, axis=2))  # (seq_length-1, n_keypoints)
    
    # Aggregate statistics
    mean_velocity = np.mean(velocity_magnitudes)
    std_velocity = np.std(velocity_magnitudes)
    max_velocity = np.max(velocity_magnitudes)
    
    return mean_velocity, std_velocity, max_velocity

def calculate_acceleration(velocity_sequence):
    """Calculate acceleration from velocity sequence"""
    # velocity_sequence shape: (seq_length-1, 33, 2)
    acceleration = np.diff(velocity_sequence, axis=0)  # (seq_length-2, 33, 2)
    return acceleration

def calculate_side_acceleration_stats(acceleration_sequence, side_keypoints):
    """Calculate acceleration statistics for a body side"""
    # acceleration_sequence shape: (seq_length-2, 33, 2)
    side_acceleration = acceleration_sequence[:, side_keypoints, :]  # (seq_length-2, n_keypoints, 2)
    
    # Calculate magnitude of acceleration vectors
    acceleration_magnitudes = np.sqrt(np.sum(side_acceleration**2, axis=2))  # (seq_length-2, n_keypoints)
    
    # Aggregate statistics
    mean_acceleration = np.mean(acceleration_magnitudes)
    std_acceleration = np.std(acceleration_magnitudes)
    max_acceleration = np.max(acceleration_magnitudes)
    
    return mean_acceleration, std_acceleration, max_acceleration

def calculate_asymmetry_features(left_stats, right_stats):
    """Calculate asymmetry ratios between left and right sides"""
    left_mean_vel, left_std_vel, left_max_vel = left_stats['velocity']
    right_mean_vel, right_std_vel, right_max_vel = right_stats['velocity']
    
    left_mean_acc, left_std_acc, left_max_acc = left_stats['acceleration']
    right_mean_acc, right_std_acc, right_max_acc = right_stats['acceleration']
    
    # Velocity asymmetry ratios
    vel_mean_ratio = left_mean_vel / (right_mean_vel + 1e-8)  # Avoid division by zero
    vel_std_ratio = left_std_vel / (right_std_vel + 1e-8)
    vel_max_ratio = left_max_vel / (right_max_vel + 1e-8)
    
    # Acceleration asymmetry ratios
    acc_mean_ratio = left_mean_acc / (right_mean_acc + 1e-8)
    acc_std_ratio = left_std_acc / (right_std_acc + 1e-8)
    acc_max_ratio = left_max_acc / (right_max_acc + 1e-8)
    
    return [vel_mean_ratio, vel_std_ratio, vel_max_ratio, 
            acc_mean_ratio, acc_std_ratio, acc_max_ratio]

def extract_dynamic_features(df):
    dynamic_features_list = []
    
    for idx, row in df.iterrows():
        skeleton_seq = np.array(row['Skeleton_Sequence'])


        skeleton_seq = skeleton_seq.reshape(skeleton_seq.shape[0], 33, 2)

        # Calculate velocity and acceleration
        velocity = calculate_velocity(skeleton_seq)
        acceleration = calculate_acceleration(velocity)
        
        # Calculate statistics for each side
        left_vel_stats = calculate_side_velocity_stats(velocity, LEFT_SIDE_KEYPOINTS)
        right_vel_stats = calculate_side_velocity_stats(velocity, RIGHT_SIDE_KEYPOINTS)
        
        left_acc_stats = calculate_side_acceleration_stats(acceleration, LEFT_SIDE_KEYPOINTS)
        right_acc_stats = calculate_side_acceleration_stats(acceleration, RIGHT_SIDE_KEYPOINTS)
        
        # Create feature dictionaries
        left_stats = {'velocity': left_vel_stats, 'acceleration': left_acc_stats}
        right_stats = {'velocity': right_vel_stats, 'acceleration': right_acc_stats}
        
        # Calculate asymmetry features
        asymmetry_features = calculate_asymmetry_features(left_stats, right_stats)
        
        # Combine all dynamic features
        dynamic_features = [
            *left_vel_stats, *right_vel_stats,
            *left_acc_stats, *right_acc_stats,
            *asymmetry_features
        ]
        
        dynamic_features_list.append(dynamic_features)
    
    return np.array(dynamic_features_list)

# Helper function to extract dynamic features for a single sequence
def extract_dynamic_features_single(skeleton_seq):
    """Extract dynamic features for a single skeleton sequence"""
    # Calculate velocity and acceleration
    velocity = calculate_velocity(skeleton_seq)
    acceleration = calculate_acceleration(velocity)
    
    # Calculate statistics for each side
    left_vel_stats = calculate_side_velocity_stats(velocity, LEFT_SIDE_KEYPOINTS)
    right_vel_stats = calculate_side_velocity_stats(velocity, RIGHT_SIDE_KEYPOINTS)
    
    left_acc_stats = calculate_side_acceleration_stats(acceleration, LEFT_SIDE_KEYPOINTS)
    right_acc_stats = calculate_side_acceleration_stats(acceleration, RIGHT_SIDE_KEYPOINTS)
    
    # Create feature dictionaries
    left_stats = {
        'velocity': left_vel_stats,
        'acceleration': left_acc_stats
    }
    right_stats = {
        'velocity': right_vel_stats,
        'acceleration': right_acc_stats
    }
    
    # Calculate asymmetry features
    asymmetry_features = calculate_asymmetry_features(left_stats, right_stats)
    
    # Combine all dynamic features
    dynamic_features = [
        *left_vel_stats, *right_vel_stats,    # 6 velocity features
        *left_acc_stats, *right_acc_stats,    # 6 acceleration features  
        *asymmetry_features                   # 6 asymmetry ratios
    ]
    
    return np.array(dynamic_features)

In [4]:
# Combined feature extraction function with top-N masking for BOTH static and dynamic features
def extract_combined_features_with_masking(df, top_n_mask):
    features_list = []
    
    for idx, row in df.iterrows():
        skeleton_seq = row['Skeleton_Sequence']
        exercise_id = row['Exercise_Id']
        
        # Get feature mask configuration for this exercise
        feature_mask_config = top_n_mask.get(exercise_id, {})
        
        # ===== EXTRACT STATIC FEATURES WITH MASKING =====
        # Ensure skeleton_seq is 3D: (seq_length, 33, 2)
        if skeleton_seq.ndim == 2 and skeleton_seq.shape[1] == 66:
            skeleton_seq = skeleton_seq.reshape(skeleton_seq.shape[0], 33, 2)
        
        # Calculate means and variances for all keypoints first
        flattened = skeleton_seq.reshape(len(skeleton_seq), -1)  # (seq_length, 66)
        all_means = np.mean(flattened, axis=0)  # 66 features
        all_variances = np.var(flattened, axis=0)  # 66 features
        
        # Apply granular masking - only keep specified static features
        final_means = np.zeros(66)
        final_variances = np.zeros(66)
        
        # For each keypoint in the mask configuration, keep specified components
        for kp_idx, components_to_keep in feature_mask_config.items():
            if kp_idx == 'dynamic':
                continue  # Skip dynamic features for now
            
            # Each keypoint has 2 positions in the mean/variance arrays
            mean_x_idx = kp_idx * 2
            mean_y_idx = kp_idx * 2 + 1
            var_x_idx = kp_idx * 2
            var_y_idx = kp_idx * 2 + 1
            
            if 'mean_x' in components_to_keep:
                final_means[mean_x_idx] = all_means[mean_x_idx]
            if 'mean_y' in components_to_keep:
                final_means[mean_y_idx] = all_means[mean_y_idx]
            if 'var_x' in components_to_keep:
                final_variances[var_x_idx] = all_variances[var_x_idx]
            if 'var_y' in components_to_keep:
                final_variances[var_y_idx] = all_variances[var_y_idx]
        
        static_features = np.concatenate([final_means, final_variances])
        
        # ===== EXTRACT DYNAMIC FEATURES WITH MASKING =====
        dynamic_features_all = extract_dynamic_features_single(skeleton_seq)
        
        # Apply masking to dynamic features
        dynamic_features_masked = np.zeros(len(dynamic_feature_names))
        if 'dynamic' in feature_mask_config:
            dynamic_indices_to_keep = feature_mask_config['dynamic']
            for idx in dynamic_indices_to_keep:
                if idx < len(dynamic_features_all):
                    dynamic_features_masked[idx] = dynamic_features_all[idx]
        
        # ===== COMBINE STATIC AND DYNAMIC FEATURES =====
        combined_features = np.concatenate([static_features, dynamic_features_masked])
        features_list.append(combined_features)
    
    return np.array(features_list)


In [5]:
# Function to create feature mask for top N features per exercise (BOTH static and dynamic)
def create_top_n_mask(n_features_per_exercise):
    mask_dict = {}
    
    for exercise in ['E1', 'E2', 'E3', 'E4', 'E5']:
        # Get top N features for this exercise (both static and dynamic)
        top_features = feature_importance_df[
            feature_importance_df['exercise'] == exercise
        ].nlargest(n_features_per_exercise, 'importance')
        
        mask_dict[exercise] = {}
        
        for _, row in top_features.iterrows():
            feature_name = row['feature']
            if feature_name in feature_to_index:
                feature_idx, feature_type = feature_to_index[feature_name]
                
                if feature_type == 'dynamic':
                    # For dynamic features, we store them with a special key
                    if 'dynamic' not in mask_dict[exercise]:
                        mask_dict[exercise]['dynamic'] = set()
                    mask_dict[exercise]['dynamic'].add(feature_idx - 132)  # Convert to dynamic feature index (0-17)
                else:
                    # For static features, use the original keypoint-based system
                    kp_idx = feature_idx
                    component = feature_type
                    if kp_idx not in mask_dict[exercise]:
                        mask_dict[exercise][kp_idx] = []
                    if component not in mask_dict[exercise][kp_idx]:
                        mask_dict[exercise][kp_idx].append(component)
    
    return mask_dict

In [6]:
# Create feature name to index mapping
keypoint_names = [
    'nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner', 'right_eye', 'right_eye_outer',
    'left_ear', 'right_ear', 'mouth_left', 'mouth_right', 'left_shoulder', 'right_shoulder',
    'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky',
    'left_index', 'right_index', 'left_thumb', 'right_thumb', 'left_hip', 'right_hip',
    'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel', 'right_heel',
    'left_foot_index', 'right_foot_index'
]

feature_to_index = {}
for i, name in enumerate(keypoint_names):
    feature_to_index[f'{name}_mean_x'] = (i, 'mean_x')
    feature_to_index[f'{name}_mean_y'] = (i, 'mean_y')
    feature_to_index[f'{name}_var_x'] = (i, 'var_x')
    feature_to_index[f'{name}_var_y'] = (i, 'var_y')

# Dynamic feature names and their indices (they come after the 132 static features)
dynamic_feature_names = [
    'left_side_mean_velocity', 'left_side_std_velocity', 'left_side_max_velocity',
    'right_side_mean_velocity', 'right_side_std_velocity', 'right_side_max_velocity',
    'left_side_mean_acceleration', 'left_side_std_acceleration', 'left_side_max_acceleration',
    'right_side_mean_acceleration', 'right_side_std_acceleration', 'right_side_max_acceleration',
    'velocity_mean_asymmetry_ratio', 'velocity_std_asymmetry_ratio', 'velocity_max_asymmetry_ratio',
    'acceleration_mean_asymmetry_ratio', 'acceleration_std_asymmetry_ratio', 'acceleration_max_asymmetry_ratio'
]

# Add dynamic features to the mapping (they start at index 132)
for i, name in enumerate(dynamic_feature_names):
    feature_to_index[name] = (132 + i, 'dynamic')

In [7]:
# Define left and right side keypoints
LEFT_SIDE_KEYPOINTS = [11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]  # Left body parts
RIGHT_SIDE_KEYPOINTS = [12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32]  # Right body parts

def calculate_velocity(sequence):
    """Calculate velocity between consecutive frames"""
    # sequence shape: (seq_length, 33, 2)
    velocity = np.diff(sequence, axis=0)  # (seq_length-1, 33, 2)
    return velocity

def calculate_side_velocity_stats(velocity_sequence, side_keypoints):
    """Calculate velocity statistics for a body side"""
    # velocity_sequence shape: (seq_length-1, 33, 2)
    side_velocity = velocity_sequence[:, side_keypoints, :]  # (seq_length-1, n_keypoints, 2)
    
    # Calculate magnitude of velocity vectors
    velocity_magnitudes = np.sqrt(np.sum(side_velocity**2, axis=2))  # (seq_length-1, n_keypoints)
    
    # Aggregate statistics
    mean_velocity = np.mean(velocity_magnitudes)
    std_velocity = np.std(velocity_magnitudes)
    max_velocity = np.max(velocity_magnitudes)
    
    return mean_velocity, std_velocity, max_velocity

def calculate_acceleration(velocity_sequence):
    """Calculate acceleration from velocity sequence"""
    # velocity_sequence shape: (seq_length-1, 33, 2)
    acceleration = np.diff(velocity_sequence, axis=0)  # (seq_length-2, 33, 2)
    return acceleration

def calculate_side_acceleration_stats(acceleration_sequence, side_keypoints):
    """Calculate acceleration statistics for a body side"""
    # acceleration_sequence shape: (seq_length-2, 33, 2)
    side_acceleration = acceleration_sequence[:, side_keypoints, :]  # (seq_length-2, n_keypoints, 2)
    
    # Calculate magnitude of acceleration vectors
    acceleration_magnitudes = np.sqrt(np.sum(side_acceleration**2, axis=2))  # (seq_length-2, n_keypoints)
    
    # Aggregate statistics
    mean_acceleration = np.mean(acceleration_magnitudes)
    std_acceleration = np.std(acceleration_magnitudes)
    max_acceleration = np.max(acceleration_magnitudes)
    
    return mean_acceleration, std_acceleration, max_acceleration

def calculate_asymmetry_features(left_stats, right_stats):
    """Calculate asymmetry ratios between left and right sides"""
    left_mean_vel, left_std_vel, left_max_vel = left_stats['velocity']
    right_mean_vel, right_std_vel, right_max_vel = right_stats['velocity']
    
    left_mean_acc, left_std_acc, left_max_acc = left_stats['acceleration']
    right_mean_acc, right_std_acc, right_max_acc = right_stats['acceleration']
    
    # Velocity asymmetry ratios
    vel_mean_ratio = left_mean_vel / (right_mean_vel + 1e-8)  # Avoid division by zero
    vel_std_ratio = left_std_vel / (right_std_vel + 1e-8)
    vel_max_ratio = left_max_vel / (right_max_vel + 1e-8)
    
    # Acceleration asymmetry ratios
    acc_mean_ratio = left_mean_acc / (right_mean_acc + 1e-8)
    acc_std_ratio = left_std_acc / (right_std_acc + 1e-8)
    acc_max_ratio = left_max_acc / (right_max_acc + 1e-8)
    
    return [vel_mean_ratio, vel_std_ratio, vel_max_ratio, 
            acc_mean_ratio, acc_std_ratio, acc_max_ratio]

def extract_dynamic_features(df):
    dynamic_features_list = []
    
    for idx, row in df.iterrows():
        skeleton_seq = np.array(row['Skeleton_Sequence'])


        skeleton_seq = skeleton_seq.reshape(skeleton_seq.shape[0], 33, 2)

        # Calculate velocity and acceleration
        velocity = calculate_velocity(skeleton_seq)
        acceleration = calculate_acceleration(velocity)
        
        # Calculate statistics for each side
        left_vel_stats = calculate_side_velocity_stats(velocity, LEFT_SIDE_KEYPOINTS)
        right_vel_stats = calculate_side_velocity_stats(velocity, RIGHT_SIDE_KEYPOINTS)
        
        left_acc_stats = calculate_side_acceleration_stats(acceleration, LEFT_SIDE_KEYPOINTS)
        right_acc_stats = calculate_side_acceleration_stats(acceleration, RIGHT_SIDE_KEYPOINTS)
        
        # Create feature dictionaries
        left_stats = {'velocity': left_vel_stats, 'acceleration': left_acc_stats}
        right_stats = {'velocity': right_vel_stats, 'acceleration': right_acc_stats}
        
        # Calculate asymmetry features
        asymmetry_features = calculate_asymmetry_features(left_stats, right_stats)
        
        # Combine all dynamic features
        dynamic_features = [
            *left_vel_stats, *right_vel_stats,
            *left_acc_stats, *right_acc_stats,
            *asymmetry_features
        ]
        
        dynamic_features_list.append(dynamic_features)
    
    return np.array(dynamic_features_list)

# Helper function to extract dynamic features for a single sequence
def extract_dynamic_features_single(skeleton_seq):
    """Extract dynamic features for a single skeleton sequence"""
    # Calculate velocity and acceleration
    velocity = calculate_velocity(skeleton_seq)
    acceleration = calculate_acceleration(velocity)
    
    # Calculate statistics for each side
    left_vel_stats = calculate_side_velocity_stats(velocity, LEFT_SIDE_KEYPOINTS)
    right_vel_stats = calculate_side_velocity_stats(velocity, RIGHT_SIDE_KEYPOINTS)
    
    left_acc_stats = calculate_side_acceleration_stats(acceleration, LEFT_SIDE_KEYPOINTS)
    right_acc_stats = calculate_side_acceleration_stats(acceleration, RIGHT_SIDE_KEYPOINTS)
    
    # Create feature dictionaries
    left_stats = {
        'velocity': left_vel_stats,
        'acceleration': left_acc_stats
    }
    right_stats = {
        'velocity': right_vel_stats,
        'acceleration': right_acc_stats
    }
    
    # Calculate asymmetry features
    asymmetry_features = calculate_asymmetry_features(left_stats, right_stats)
    
    # Combine all dynamic features
    dynamic_features = [
        *left_vel_stats, *right_vel_stats,    # 6 velocity features
        *left_acc_stats, *right_acc_stats,    # 6 acceleration features  
        *asymmetry_features                   # 6 asymmetry ratios
    ]
    
    return np.array(dynamic_features)

In [8]:
# Combined feature extraction function with top-N masking for BOTH static and dynamic features
def extract_combined_features_with_masking(df, top_n_mask):
    features_list = []
    
    for idx, row in df.iterrows():
        skeleton_seq = row['Skeleton_Sequence']
        exercise_id = row['Exercise_Id']
        
        # Get feature mask configuration for this exercise
        feature_mask_config = top_n_mask.get(exercise_id, {})
        
        # ===== EXTRACT STATIC FEATURES WITH MASKING =====
        # Ensure skeleton_seq is 3D: (seq_length, 33, 2)
        if skeleton_seq.ndim == 2 and skeleton_seq.shape[1] == 66:
            skeleton_seq = skeleton_seq.reshape(skeleton_seq.shape[0], 33, 2)
        
        # Calculate means and variances for all keypoints first
        flattened = skeleton_seq.reshape(len(skeleton_seq), -1)  # (seq_length, 66)
        all_means = np.mean(flattened, axis=0)  # 66 features
        all_variances = np.var(flattened, axis=0)  # 66 features
        
        # Apply granular masking - only keep specified static features
        final_means = np.zeros(66)
        final_variances = np.zeros(66)
        
        # For each keypoint in the mask configuration, keep specified components
        for kp_idx, components_to_keep in feature_mask_config.items():
            if kp_idx == 'dynamic':
                continue  # Skip dynamic features for now
            
            # Each keypoint has 2 positions in the mean/variance arrays
            mean_x_idx = kp_idx * 2
            mean_y_idx = kp_idx * 2 + 1
            var_x_idx = kp_idx * 2
            var_y_idx = kp_idx * 2 + 1
            
            if 'mean_x' in components_to_keep:
                final_means[mean_x_idx] = all_means[mean_x_idx]
            if 'mean_y' in components_to_keep:
                final_means[mean_y_idx] = all_means[mean_y_idx]
            if 'var_x' in components_to_keep:
                final_variances[var_x_idx] = all_variances[var_x_idx]
            if 'var_y' in components_to_keep:
                final_variances[var_y_idx] = all_variances[var_y_idx]
        
        static_features = np.concatenate([final_means, final_variances])
        
        # ===== EXTRACT DYNAMIC FEATURES WITH MASKING =====
        dynamic_features_all = extract_dynamic_features_single(skeleton_seq)
        
        # Apply masking to dynamic features
        dynamic_features_masked = np.zeros(len(dynamic_feature_names))
        if 'dynamic' in feature_mask_config:
            dynamic_indices_to_keep = feature_mask_config['dynamic']
            for idx in dynamic_indices_to_keep:
                if idx < len(dynamic_features_all):
                    dynamic_features_masked[idx] = dynamic_features_all[idx]
        
        # ===== COMBINE STATIC AND DYNAMIC FEATURES =====
        combined_features = np.concatenate([static_features, dynamic_features_masked])
        features_list.append(combined_features)
    
    return np.array(features_list)


In [9]:
# Function to create feature mask for top N features per exercise (BOTH static and dynamic)
def create_top_n_mask(n_features_per_exercise):
    mask_dict = {}
    
    for exercise in ['E1', 'E2', 'E3', 'E4', 'E5']:
        # Get top N features for this exercise (both static and dynamic)
        top_features = feature_importance_df[
            feature_importance_df['exercise'] == exercise
        ].nlargest(n_features_per_exercise, 'importance')
        
        mask_dict[exercise] = {}
        
        for _, row in top_features.iterrows():
            feature_name = row['feature']
            if feature_name in feature_to_index:
                feature_idx, feature_type = feature_to_index[feature_name]
                
                if feature_type == 'dynamic':
                    # For dynamic features, we store them with a special key
                    if 'dynamic' not in mask_dict[exercise]:
                        mask_dict[exercise]['dynamic'] = set()
                    mask_dict[exercise]['dynamic'].add(feature_idx - 132)  # Convert to dynamic feature index (0-17)
                else:
                    # For static features, use the original keypoint-based system
                    kp_idx = feature_idx
                    component = feature_type
                    if kp_idx not in mask_dict[exercise]:
                        mask_dict[exercise][kp_idx] = []
                    if component not in mask_dict[exercise][kp_idx]:
                        mask_dict[exercise][kp_idx].append(component)
    
    return mask_dict

In [10]:
def run_random_cv(model, X_combined, y_sequences, sequence_patient_ids, 
                 patient_to_label, patient_ids, Y_train_filtered, n_splits=100):
    """Run random leave-3-out cross-validation with n_splits random combinations"""
    majority_accuracies = []
    probability_accuracies = []
    
    # Generate all possible test combinations
    all_test_combinations = list(combinations(patient_ids, 3))
    
    # Randomly select n_splits combinations
    rng = np.random.RandomState(42)
    selected_combinations = rng.choice(len(all_test_combinations), 
                                     size=min(n_splits, len(all_test_combinations)), 
                                     replace=False)
    
    for idx in tqdm(selected_combinations, desc=f"Random CV ({n_splits} splits)", leave=False):
        test_patients_tuple = all_test_combinations[idx]
        test_patients = np.array(test_patients_tuple)
        train_patients = np.setdiff1d(patient_ids, test_patients)
        
        # Create masks
        train_mask = np.isin(sequence_patient_ids, train_patients)
        test_mask = np.isin(sequence_patient_ids, test_patients)
        
        # Split data
        X_train_split = X_combined[train_mask]
        X_test_split = X_combined[test_mask]
        y_train_split = y_sequences[train_mask]
        test_patient_ids_split = sequence_patient_ids[test_mask]
        
        # Train model
        model.fit(X_train_split, y_train_split)
        
        # Get predictions
        test_sequence_preds = model.predict(X_test_split)
        test_sequence_probs = model.predict_proba(X_test_split)
        
        # Aggregate to patient level
        majority_acc, probability_acc = aggregate_patient_predictions(
            test_sequence_preds, test_sequence_probs, test_patient_ids_split, 
            test_patients, patient_to_label
        )
        
        majority_accuracies.append(majority_acc)
        probability_accuracies.append(probability_acc)
    
    return majority_accuracies, probability_accuracies

def aggregate_patient_predictions(test_sequence_preds, test_sequence_probs, 
                                 test_patient_ids_split, test_patients, patient_to_label):
    """Aggregate sequence predictions to patient level using both methods"""
    # Majority Voting
    patient_predictions = {}
    for i, patient_id in enumerate(test_patient_ids_split):
        if patient_id not in patient_predictions:
            patient_predictions[patient_id] = []
        patient_predictions[patient_id].append(test_sequence_preds[i])
    
    majority_preds = []
    true_labels = []
    for patient_id in test_patients:
        votes = patient_predictions[patient_id]
        pred_label = np.argmax(np.bincount(votes))
        majority_preds.append(pred_label)
        true_labels.append(patient_to_label[patient_id])
    
    majority_acc = balanced_accuracy_score(true_labels, majority_preds)
    
    # Probability Averaging
    patient_probs = {}
    for i, patient_id in enumerate(test_patient_ids_split):
        if patient_id not in patient_probs:
            patient_probs[patient_id] = []
        patient_probs[patient_id].append(test_sequence_probs[i])
    
    probability_preds = []
    for patient_id in test_patients:
        avg_probs = np.mean(patient_probs[patient_id], axis=0)
        pred_label = np.argmax(avg_probs)
        probability_preds.append(pred_label)
    
    probability_acc = balanced_accuracy_score(true_labels, probability_preds)
    
    return majority_acc, probability_acc

In [ ]:
def run_rf_hyperparameter_search():
    # Configuration
    TOP_FEATURES = 18
    N_SPLITS = 50  # Use 50 random splits instead of exhaustive
    print(f"Running Random Forest hyperparameter optimization with top {TOP_FEATURES} features...")
    print(f"Using {N_SPLITS} random splits for faster evaluation")
    
    # Load data
    print("Loading data...")
    X_train = joblib.load("Data/Xtrain2.pkl")
    Y_train = np.load('Data/Ytrain2.npy')
    
    # Get patient IDs and create mappings
    patient_ids = np.sort(X_train['Patient_Id'].unique())
    all_patient_to_label = dict(zip(range(1, 15), Y_train))
    patient_to_label = {pid: all_patient_to_label[pid] for pid in patient_ids}
    Y_train_filtered = np.array([patient_to_label[pid] for pid in patient_ids])
    
    print(f"Data loaded: {X_train.shape[0]} sequences, {len(patient_ids)} patients")
    
    # Load feature importance and create mask for top N features
    print("Loading feature importance and creating feature mask...")
    feature_importance_df = pd.read_csv('feature_importance_v2_baseline.csv')
    top_n_mask = create_top_n_mask(TOP_FEATURES)
    
    # Extract features
    print("Extracting features...")
    X_combined_features = extract_combined_features_with_masking(X_train, top_n_mask)
    
    # Add exercise encoding
    from sklearn.preprocessing import OneHotEncoder
    exercise_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    exercise_encoded = exercise_encoder.fit_transform(X_train[['Exercise_Id']])
    X_combined = np.concatenate([X_combined_features, exercise_encoded], axis=1)
    
    # Create sequence-level labels and patient IDs
    y_sequences = X_train['Patient_Id'].map(patient_to_label).values
    sequence_patient_ids = X_train['Patient_Id'].values
    
    print(f"Final feature matrix shape: {X_combined.shape}")
    
    # Define SMALLER Random Forest hyperparameter grid
    param_grid = [
        {
            'rf__n_estimators': [50, 100],  # Reduced from 3 to 2 options
            'rf__max_depth': [5, 10, None],  # Reduced from 5 to 3 options
            'rf__min_samples_split': [2, 5],  # Reduced from 3 to 2 options
            'rf__min_samples_leaf': [1, 2],   # Reduced from 3 to 2 options
            'rf__max_features': ['sqrt', 'log2']  # Removed None, kept 2 options
        }
    ]
    
    # Generate all parameter combinations
    from sklearn.model_selection import ParameterGrid
    all_params = list(ParameterGrid(param_grid))
    print(f"Testing {len(all_params)} hyperparameter combinations (reduced grid)...")
    
    # Results storage
    results = []
    
    # Test each hyperparameter combination
    for params in tqdm(all_params, desc="Random Forest hyperparameter search"):
        # Create pipeline with current parameters
        pipeline = Pipeline([
            # Remove scaler for Random Forest (not needed and saves computation)
            ('rf', RandomForestClassifier(
                n_estimators=params['rf__n_estimators'],
                max_depth=params['rf__max_depth'],
                min_samples_split=params['rf__min_samples_split'],
                min_samples_leaf=params['rf__min_samples_leaf'],
                max_features=params['rf__max_features'],
                class_weight='balanced',
                random_state=42,
                n_jobs=-1  # Use all cores
            ))
        ])
        
        # Run RANDOM cross-validation (100 splits instead of exhaustive)
        majority_accuracies, probability_accuracies = run_random_cv(
            pipeline, X_combined, y_sequences, sequence_patient_ids, 
            patient_to_label, patient_ids, Y_train_filtered, n_splits=N_SPLITS
        )
        
        # Store results
        result = {
            'n_estimators': params['rf__n_estimators'],
            'max_depth': params['rf__max_depth'],
            'min_samples_split': params['rf__min_samples_split'],
            'min_samples_leaf': params['rf__min_samples_leaf'],
            'max_features': params['rf__max_features'],
            'majority_mean': np.mean(majority_accuracies),
            'majority_std': np.std(majority_accuracies),
            'majority_min': np.min(majority_accuracies),
            'majority_max': np.max(majority_accuracies),
            'probability_mean': np.mean(probability_accuracies),
            'probability_std': np.std(probability_accuracies),
            'probability_min': np.min(probability_accuracies),
            'probability_max': np.max(probability_accuracies)
        }
        results.append(result)
    
    # Convert to DataFrame and find best models
    results_df = pd.DataFrame(results)
    
    # Find best parameters for each method
    best_majority = results_df.loc[results_df['majority_mean'].idxmax()]
    best_probability = results_df.loc[results_df['probability_mean'].idxmax()]
    
    # Print results
    print("\n" + "="*80)
    print("RANDOM FOREST HYPERPARAMETER OPTIMIZATION RESULTS")
    print("="*80)
    
    print(f"\nBEST MAJORITY VOTING MODEL:")
    print(f"  Parameters: n_estimators={best_majority['n_estimators']}, "
          f"max_depth={best_majority['max_depth']}, "
          f"min_samples_split={best_majority['min_samples_split']}, "
          f"min_samples_leaf={best_majority['min_samples_leaf']}, "
          f"max_features={best_majority['max_features']}")
    print(f"  Performance: {best_majority['majority_mean']:.4f} (±{best_majority['majority_std']:.4f})")
    print(f"  Range: [{best_majority['majority_min']:.4f}, {best_majority['majority_max']:.4f}]")
    
    print(f"\nBEST PROBABILITY AVERAGING MODEL:")
    print(f"  Parameters: n_estimators={best_probability['n_estimators']}, "
          f"max_depth={best_probability['max_depth']}, "
          f"min_samples_split={best_probability['min_samples_split']}, "
          f"min_samples_leaf={best_probability['min_samples_leaf']}, "
          f"max_features={best_probability['max_features']}")
    print(f"  Performance: {best_probability['probability_mean']:.4f} (±{best_probability['probability_std']:.4f})")
    print(f"  Range: [{best_probability['probability_min']:.4f}, {best_probability['probability_max']:.4f}]")
    
    return results_df, best_majority, best_probability

In [12]:
# Run the optimization
if __name__ == "__main__":
    results_df, best_majority, best_probability = run_rf_hyperparameter_search()

Running Random Forest hyperparameter optimization with top 18 features...
Using 50 random splits for faster evaluation
Loading data...


Data loaded: 444 sequences, 14 patients
Loading feature importance and creating feature mask...
Extracting features...
Final feature matrix shape: (444, 155)
Testing 48 hyperparameter combinations (reduced grid)...


Random Forest hyperparameter search: 100%|██████████| 48/48 [09:16<00:00, 11.60s/it]


RANDOM FOREST HYPERPARAMETER OPTIMIZATION RESULTS

BEST MAJORITY VOTING MODEL:
  Parameters: n_estimators=100, max_depth=5.0, min_samples_split=2, min_samples_leaf=2, max_features=sqrt
  Performance: 0.7600 (±0.2481)
  Range: [0.2500, 1.0000]

BEST PROBABILITY AVERAGING MODEL:
  Parameters: n_estimators=100, max_depth=5.0, min_samples_split=2, min_samples_leaf=2, max_features=sqrt
  Performance: 0.7667 (±0.2500)
  Range: [0.2500, 1.0000]

Detailed results saved to: rf_hyperparameter_results_top18.csv
